In [1]:
import python.data as data
project_folder = "monresovelo"

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pickle 
import pandas as pd

token = "YOUR MAPBOX TOKEN"


## Observations (GPS tracks)

Check if the observations have already been preprocessed, if not, launch a python script that do it.

<font color='red'> <b>The project_folder variable cannot be used after %run so if using another project than monresovelo it needs to be manually modified.</b></font>

<font color='green'>The preprocessed observations are stored in <b>files/PROJECT_FOLDER/data_processed/observations_matched.df</b>.</font>


In [2]:
if(not data.check_file("files/"+project_folder+"/data_processed/observations_matched.df")):
    %run data/monresovelo/load_data.py -h

Loading data...
4880th route 


Simplify the observations, trying to reduce the number of GPS points of each one. The function automatically check if it has already been done or not. 

<font color='green'>The simplified observations are stored in <b>files/PROJECT_FOLDER/data_processed/observations_matched_simplified.df</b></font>

In [3]:
data.simplify_gps("files/"+project_folder+"/data_processed/observations_matched.df", 
              "files/"+project_folder+"/data_processed/observations_matched_simplified.df")

Simplifying route 4880/4880 

## Shortest paths

Check if a road graph has already been loaded and saved, if not, launch a python script that load and save the road graph containing all the observations previously preprocessed.

<font color='green'>The road graph is stored in <b>files/PROJECT_FOLDER/city_graphs/city.ox</b></font>

In [4]:
if(not data.check_file("files/"+project_folder+"/city_graphs/city.ox")):
    %run data/monresovelo/load_city.py -h

Creating city.ox, this might take several minutes...


Use the road graph to find the shortest path between the origin and the destination point of each simplified observations. 

<font color='green'>The shortest paths are stored in <b>files/PROJECT_FOLDER/data_processed/osmnx_pathfinding.df</b>.</font>

In [5]:
data.pathfinding_osmnx("files/"+project_folder+"/data_processed/observations_matched_simplified.df", 
                       "files/"+project_folder+"/data_processed/osmnx_pathfinding.df", 
                       "files/"+project_folder+"/city_graphs/city.ox")

Finding the shortest path for route 4880/4880 with OSMNX.

<font color='green'>The simplified shortest paths are stored in <b>files/PROJECT_FOLDER/data_processed/osmnx_pathfinding_simplified.df</b>.</font>

In [6]:
data.simplify_gps("files/"+project_folder+"/data_processed/osmnx_pathfinding.df", 
                  "files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df")

Simplifying route 4880/4880 

## Distances

Compute the distance of each observation and each shortest paths. 

<font color='green'>The distances are stored in <b>files/PROJECT_FOLDER/distances/</b>.</font>

In [7]:
data.compute_distance("files/"+project_folder+"/data_processed/observations_matched_simplified.df",
                      "files/"+project_folder+"/distances/distances_observations.tab")
data.compute_distance("files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df", 
                      "files/"+project_folder+"/distances/distances_osmnx.tab")

Calculating the distance of route 4880/4880.
Calculating the distance of route 4880/4880.


## Mapbox

Send a request to Mapbox API to find a cycling route between the origin and destination points of each observation.

<font color='red'><b>
This tile will throw a "KeyError: 'code'" because of the limited number of requests that can be send to the mapbox API. When it happens, just wait a few seconds and launch it again until all the routes have been computed.</b></font>

<font color='green'>The Mapbox routes are stored in <b>files/PROJECT_FOLDER/data_processed/mapbox_pathfinding.df</b>.</font>

In [22]:
data.pathfinding_mapbox("files/"+project_folder+"/data_processed/observations_matched_simplified.df", 
                    "files/"+project_folder+"/data_processed/mapbox_pathfinding.df", token)

Finding the shortest path for route 4880/4880 using Mapbox.

<font color='green'>The simplified Mapbox routes are stored in <b>files/PROJECT_FOLDER/data_processed/mapbox_pathfinding_simplified.df</b>.</font>

In [23]:
data.simplify_gps("files/"+project_folder+"/data_processed/mapbox_pathfinding.df", 
                  "files/"+project_folder+"/data_processed/mapbox_pathfinding_simplified.df")

Simplifying route 4880/4880 

<b>Once all the data has been created for a project, you can use the notebook main.ipynb.</b>